# Name and Address Parsing
> Recognising Person names and Addresses in a text using NER and NLP modeling techniques

## Get data from MongoDB

In [ ]:
!pip install pymongo[tls,srv]

In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
%reload_ext google.colab.data_table

In [ ]:
import pymongo 
import pprint
mongo_uri = "mongodb+srv://<userid>:<pass>@<server>.azure.mongodb.net/<db>?retryWrites=true&w=majority"
client = pymongo.MongoClient(mongo_uri)

In [ ]:
listdb = client.list_database_names(); listdb

['testdb']

In [ ]:
db = client['testdb']
db.list_collection_names()

['entities', 'parcels', 'taxes']

In [ ]:
# print(db.command("collstats", "events"))
db.command("dbstats")

{'$clusterTime': {'clusterTime': Timestamp(1598254711, 1),
  'signature': {'hash': b'\x19\x92\xd6\x94\x06\xed\xa9\xb5s+\x8e\xcaqn\xb3J\x13_\x1e\xab',
   'keyId': 6839689286636273667}},
 'avgObjSize': 597.6787450874097,
 'collections': 3,
 'dataSize': 5494893423.0,
 'db': 'testdb',
 'fsTotalSize': 34342961152.0,
 'fsUsedSize': 11420770304.0,
 'indexSize': 378994688.0,
 'indexes': 9,
 'numExtents': 0,
 'objects': 9193724,
 'ok': 1.0,
 'operationTime': Timestamp(1598254711, 1),
 'scaleFactor': 1.0,
 'storageSize': 2160832512.0,
 'views': 0}

In [ ]:
# collection1 = db['entities']
# df1 = pd.DataFrame(list(collection1.find()))
# print(df1.info())
# # print(df1.describe())
# df1.sample(5)

In [ ]:
# df1.to_pickle('/content/drive/My Drive/df_entities.p')
# del df1
# import gc
# gc.collect()

258

In [ ]:
collection2 = db['parcels']
cursor = collection2.find()
max = 0
maxL = {}
for i in range(cnt):
  xx = next(cursor)
  if len(xx)>max:
    max = len(xx)
    maxL = xx
max

46

In [ ]:
cnt = collection2.count()

In [ ]:
list(maxL)

4597

In [ ]:
collection2 = db['parcels']
cursor = collection2.find()
XX = pd.DataFrame(columns=list(maxL))
XX.to_csv('df_parcels.csv', index=False)
for i in tqdm(range((cnt//10000)+1)):
  YY = {}
  for j in range(10000):
    YY[j] = next(cursor)
  pd.DataFrame(YY).T.to_csv('df_parcels.csv', mode='a', index=False, header=False)
# pd.DataFrame(next(cursor), index=[j]).to_csv('df_parcels.csv', mode='a', index=False, header=False)

 33%|███▎      | 150/460 [04:52<10:09,  1.97s/it]

In [ ]:
pd.read_csv('df_parcels.csv')

In [ ]:
collection2 = db['parcels']
df2 = pd.DataFrame(list(collection2.find()))
print(df2.info())
# print(df2.describe())
# df2.sample(5)

## Data wrangling

In [ ]:
!pip install -q probablepeople
!pip install -q usaddress

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import  probablepeople as pp
import usaddress as ua
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()
%reload_ext autoreload
%autoreload 2
%reload_ext google.colab.data_table
%config InlineBackend.figure_format = 'retina'

plt.style.use('fivethirtyeight')
plt.style.use('seaborn-notebook')

In [ ]:
df = pd.read_pickle(os.path.join(path,'sample.p'))
df.info()
data = df.oor.tolist()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 3605089 to 2516200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     1000 non-null   object
 1   oor     1000 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


In [ ]:
data[:10]

In [ ]:
combos = [' US ',' USA ',' U.S. ',' U.S.A. ']
def patch_country(text):
  cflagz = ''
  otext = text
  text = text[-10:]
  text = ' ' + text + ' '
  for i in combos:
    if i in text:
      cflagz = i
  text = otext[:-10] + re.sub(cflagz, '', text)
  return text, cflagz

In [ ]:
def patch_household(xz):
  xx = pd.DataFrame(pp.tag(xz)[0], index=[0])
  xx['Type'] = 'Person'
  if 'And' in xx.columns.tolist():
    xx = pd.DataFrame({'Household':xz}, index=[0])
    xx['Type'] = 'Household'
  return xx

In [ ]:
tags = pp.LABELS
tags.extend(ua.LABELS)
tags = list(set(tags))
additionals = ['CountryName', 'Household', 'Type', 'NameConfidence', 'AddrConfidence', 'Text']
tags.extend(additionals)

In [ ]:
dfx = pd.DataFrame(columns=tags)
dfx = dfx.loc[:,~dfx.columns.duplicated()]

In [ ]:
# def temp(text):
#   # replace long country name with short one
#   text = re.sub('UNITED STATES OF AMERICA','US',text)
#   # country patch
#   text, cflag = patch_country(text)
#   # try and catch
#   try:
#     df1 = pd.DataFrame(ua.tag(text)[0], index=[0])
#     xz = df1.Recipient.values[0]
#     df2 = patch_household(xz)
#     xx = pd.concat([df1,df2], axis=1)
#     xx['Confidence'] = ua.tag(text)[1]
#   except:
#     try:
#       df2 = patch_household(text)
#       xx = df2
#       xx['Confidence'] = 'AddressError'
#     except:
#       xx = pd.DataFrame({'Recipient':text}, index=[0])
#       xx['Confidence'] = 'PersonError'
#   # add country label
#   if cflag!='':
#     xx['CountryName'] = cflag
    
#   return xx#.T.to_dict()[0]

In [ ]:
def temp(text):
  # country patch
  text = re.sub('UNITED STATES OF AMERICA','US',text)
  text, cflag = patch_country(text)
  # address parsing
  try:
    df1 = pd.DataFrame(ua.tag(text)[0], index=[0])
    df1['AddrConfidence'] = ua.tag(text)[1]
  except:
    df1 = pd.DataFrame(ua.parse(text)).groupby(1).agg({0: lambda x: ' '.join(x)}).T
    df1['AddrConfidence'] = 'Error'
  # address to name linking
  try:
    xz = df1.Recipient.values[0]
  except:
    xz = text
  # name parsing
  try:
    df2 = pd.DataFrame(pp.tag(xz)[0], index=[0])
    df2['NameConfidence'] = pp.tag(xz)[1]
  except:
    df2 = pd.DataFrame(pp.parse(xz)).groupby(1).agg({0: lambda x: ' '.join(x)}).T
    df2['NameConfidence'] = 'Error'
  # person name patch
  df2['Type'] = 'Person'
  if 'MiddleName' in df2.columns.tolist():
    gname, mname, sname = df2.MiddleName, df2.Surname, df2.GivenName
    df2.MiddleName, df2.Surname, df2.GivenName = mname[0], sname[0], gname[0]
  elif 'GivenName' in df2.columns.tolist():
    gname, sname = df2.Surname, df2.GivenName
    df2.Surname, df2.GivenName = sname[0], gname[0]
  # household name patch
  if 'And' in df2.columns.tolist():
    df2 = pd.DataFrame({'Household':xz}, index=[0])
    df2['Type'] = 'Household'
  # concatenation
  df = pd.concat([df1,df2], axis=1)
  df['Text'] = text  
  df['CountryName'] = cflag if cflag!='' else np.NaN
  try:
    df.loc[pd.notnull(df['CorporationName']), 'Type'] = 'Corporation'
  except:
    pass
  return df

In [ ]:
errs = {}
dfx = pd.DataFrame(columns=tags)
dfx = dfx.loc[:,~dfx.columns.duplicated()]
for idx, text in tqdm(enumerate(data)):
  try:
    dfx = dfx.append(temp(text))
    dfx.loc[pd.notnull(dfx['CorporationName']), 'Type'] = 'Corporation'
  except Exception as e:
    errs[idx] = e
    pass
dfx = dfx.fillna('')

1000it [00:18, 52.66it/s]


In [ ]:
err_df = pd.DataFrame(errs, index=[0]).T
err_df = err_df.merge(pd.Series(data, name='text'), left_index=True, right_index=True)
err_df.columns = ['error','text']
err_df = err_df[['text','error']]
err_df.iloc[:]

In [ ]:
dfbackup = dfx.copy()

In [ ]:
dfx = dfbackup[tags]
dfx.drop(['Recipient', 'CountryName'], axis=1, inplace=True)
dfx = dfx.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all')
dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 0 to 0
Data columns (total 39 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   USPSBoxType                      59 non-null     object
 1   CorporationCommitteeType         1 non-null      object
 2   SubaddressType                   18 non-null     object
 3   ZipCode                          703 non-null    object
 4   GivenName                        417 non-null    object
 5   StreetNamePreDirectional         86 non-null     object
 6   CorporationNameAndCompany        4 non-null      object
 7   StreetName                       906 non-null    object
 8   CorporationName                  381 non-null    object
 9   PlaceName                        888 non-null    object
 10  StateName                        807 non-null    object
 11  ShortForm                        1 non-null      object
 12  Surname                          407 n

In [ ]:
def func(text):
  text = str(text)
  xx = temp(text)
  xx = xx.drop(['AddrConfidence','NameConfidence','Text'], axis=1)
  try:
    xx = xx.drop(['Recipient'], axis=1)
  except:
    pass
  xx = xx.T.to_dict()[0]
  return str(xx)

In [ ]:
# !pip install -q gradio
# import gradio as gr
gr.Interface(fn=func, inputs="text", outputs="text").launch()

## Sampling

In [ ]:
path = '.'

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
tqdm.pandas()
warnings.filterwarnings("ignore")

In [ ]:
# df = pd.read_pickle(os.path.join(path,'df_entities.p'))
# df.info()

In [ ]:
# def process_text(text):
#   text = re.sub('"','',text)
#   text = re.sub(',','',text)
#   text = ' '.join(text.split())
#   return text

# sampledf = df[['oor']].sample(5000, random_state=40)
# sampledf['oor'] = sampledf.oor.apply(process_text)
# sampledf = sampledf.replace(r'^\s*$', np.nan, regex=True).dropna()
# msk = np.random.rand(len(sampledf)) < 0.8
# sampledf[msk].to_csv('train_28118_21.csv', index=False, header=None)
# sampledf[~msk].to_csv('test_28118_21.csv', index=False, header=None)

In [ ]:
!pip install doccano-transformer

import spacy
import random
from doccano_transformer.datasets import NERDataset
from doccano_transformer.utils import read_jsonl
dataset = read_jsonl(filepath='file.json1', dataset=NERDataset, encoding='utf-8')
dataset.to_spacy(tokenizer=str.split)

TRAIN_DATA = []
for x in dataset:
  xx = {}
  try:
    xx['entities'] = list(x.labels.values())[0]
    TRAIN_DATA.append((x.text.lower(), xx))
  except:
    pass

In [ ]:
def train_spacy(data, iterations, model=None):
    TRAIN_DATA = data
    if model is not None:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")
        print("Created blank 'en' model") 
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.1,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

prdnlp = train_spacy(TRAIN_DATA, 200)

Created blank 'en' model
Statring iteration 0
{'ner': 3007.4073503919476}
Statring iteration 1
{'ner': 1640.818005657745}
Statring iteration 2
{'ner': 1289.2707772499139}
Statring iteration 3
{'ner': 1141.5257107774235}
Statring iteration 4
{'ner': 915.537918427113}
Statring iteration 5
{'ner': 880.0738022538334}
Statring iteration 6
{'ner': 697.9718340145506}
Statring iteration 7
{'ner': 477.89584190603824}
Statring iteration 8
{'ner': 450.35748743359136}
Statring iteration 9
{'ner': 408.3674275620642}
Statring iteration 10
{'ner': 379.27579749970243}
Statring iteration 11
{'ner': 346.4166141593435}
Statring iteration 12
{'ner': 294.0780766001076}
Statring iteration 13
{'ner': 312.39254089213495}
Statring iteration 14
{'ner': 306.54315512528}
Statring iteration 15
{'ner': 394.02619706473706}
Statring iteration 16
{'ner': 329.8686837863413}
Statring iteration 17
{'ner': 243.71909372259998}
Statring iteration 18
{'ner': 201.99860228650502}
Statring iteration 19
{'ner': 249.2113459896862

In [ ]:
# Save our trained Model
# modelfile = input("Enter your Model Name: ")
# prdnlp.to_disk(modelfile)

prdnlp.to_disk('ner233')
import shutil
shutil.make_archive(os.path.join(path,'ner233'),'zip','/content/ner233')

In [ ]:
# test = pd.read_csv('/content/test_28118_21.csv', header=None)
# test.head()

In [ ]:
# from spacy.gold import GoldParse
# from spacy.scorer import Scorer

# def evaluate(ner_model, examples):
#     scorer = Scorer()
#     for input_, annot in examples:
#         doc_gold_text = ner_model.make_doc(input_)
#         gold = GoldParse(doc_gold_text, entities=annot)
#         pred_value = ner_model(input_)
#         scorer.score(pred_value, gold)
#     return scorer.scores

# examples = [
#     ('Who is Shaka Khan?',
#      [(7, 17, 'PERSON')]),
#     ('I like London and Berlin.',
#      [(7, 13, 'LOC'), (18, 24, 'LOC')])
# ]

# results = evaluate(prdnlp, examples)

# def report_scores(scores, e):
#     """
#     prints precision recall and f_measure
#     :param scores:
#     :return:
#     """

#     precision = '%.2f' % scores['ents_p']
#     recall = '%.2f' % scores['ents_r']
#     f_measure = '%.2f' % scores['ents_f']
#     print('%-25s %-10s %-10s %-10s' % (e, precision, recall, f_measure))

# report_scores(results, 'x')

x                         0.00       0.00       0.00      


In [ ]:
# Household -> Pass again for name tags

In [ ]:
tags = ['EntityType','Recipient','Address',
        'GivenName','MiddleName','SurName','Household','Corporation',
        'StreetAddress','City','State','Zipcode','Country']

In [ ]:
PAD_TAIL = '<redacted>'

In [ ]:
def household_patch(row):
  if row.EntityType=='Household':
    text = row.Household.split('&')[0] if '&' in row.Household else ' '.join(row.Household.split()[:2])
    text = parseit(text+PAD_TAIL)
    row['GivenName'] = text.loc[0,'GivenName']
    row['SurName'] = text.loc[0,'SurName']
    row['MiddleName'] = text.loc[0,'MiddleName']
  return row

In [ ]:
def add_on(xx):
  # Address is the combination of street, city, state, zip and country
  xx['Address'] = xx['StreetAddress'] +' '+ xx['City'] +' '+ xx['State'] +' '+ xx['Zipcode'] +' '+ xx['Country']
  # default is person, if household column is not empty, then household, same for corporation
  xx['EntityType'] = 'Person'
  xx.loc[xx.Household!='','EntityType'] = 'Household'
  xx.loc[xx.Corporation!='','EntityType'] = 'Corporation'
  # default is full name of person, if entity is corporation ,then corporation name, same for household
  xx['Recipient'] = xx['SurName'] +' '+ xx['GivenName'] +' '+ xx['MiddleName']
  xx['Recipient'] = xx.apply(lambda row: row.Corporation if row.EntityType=='Corporation' else row.Recipient, axis=1)
  xx['Recipient'] = xx.apply(lambda row: row.Household if row.EntityType=='Household' else row.Recipient, axis=1)
  # adding household to name field patch
  xx = xx.apply(household_patch, axis=1)
  # converting to dictionary format
  xx = xx.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='any')
  xx = xx.T.to_dict()[0]
  # return the processed data
  return xx

In [ ]:
def parseit(text):
  text = str(text).upper()
  text = re.sub('"','',text)
  text = re.sub(',','',text)
  text = ' '.join(text.split())
  output = {}
  doc = prdnlp(text)
  for ent in doc.ents:
    output[ent.label_] = ent.text
  
  df = pd.DataFrame(output, index=[0])
  dfx = pd.DataFrame(columns=tags)
  dfx = dfx.append(df).fillna('')
  
  return dfx

In [ ]:
def func(text):
  X = parseit(text)
  X = add_on(X)
  X = str(X)
  return X

In [ ]:
# text = test[0].sample().tolist()[0]
text = '<redacted>'
# xx = parseit(text); xx
# xx = pd.DataFrame(xx, index=[0]).T.reset_index(); xx.columns = ['Tag','Entity']; xx

In [ ]:
func('<redacted>')

In [ ]:
# !pip install -q gradio
# import gradio as gr
# inputs = gr.inputs.Textbox(lines=3, label='Input')
# outputs = gr.outputs.Textbox(label='Output')
gr.Interface(fn=func, inputs=inputs, outputs=outputs).launch()

## Flair NER Model

In [ ]:
!pip install doccano-transformer
from doccano_transformer.datasets import NERDataset
from doccano_transformer.utils import read_jsonl
dataset = read_jsonl(filepath='file_2.json1', dataset=NERDataset, encoding='utf-8')
dataset.to_spacy(tokenizer=str.split)

In [ ]:
TRAIN_DATA = []
for x in dataset:
  xx = {}
  try:
    xx['entities'] = list(x.labels.values())[0]
    TRAIN_DATA.append((x.text, xx))
  except:
    pass

In [ ]:
import spacy
from spacy.gold import biluo_tags_from_offsets
nlp = spacy.load('en_core_web_sm')
docs = []
for text, annot in TRAIN_DATA:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot['entities'])
    tags = [x.replace('U-','B-') for x in tags]
    tags = [x.replace('L-','I-') for x in tags]
    tmpdocs = [(x,y) for x,y in zip(text.split(),tags)]
    docs.append(tmpdocs)

In [ ]:
import random
import pandas as pd
X = pd.DataFrame(docs[0])
X['break'] = 'train'
X = X.append(pd.Series(dtype='str'), ignore_index=True)
for x in docs[1:]:
  Xt = pd.DataFrame(x)
  Xt = Xt.append(pd.Series(dtype='str'), ignore_index=True) 
  Xt['break'] = random.choices(['train','val','test'], weights=(90,5,5), k=1)[0]
  X = X.append(Xt)
X = X.fillna('')
X = X.reset_index(drop=True)

train = X.loc[X['break']=='train',[0,1]]; train.to_csv('train.txt', sep=' ', header=None, index=False)
val = X.loc[X['break']=='val',[0,1]]; val.to_csv('val.txt', sep=' ', header=None, index=False)
test = X.loc[X['break']=='test',[0,1]]; test.to_csv('test.txt', sep=' ', header=None, index=False)

In [ ]:
import re
import numpy as np

In [ ]:
!pip install -q flair
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {0:'text', 1:'ner'}

data_folder = '/content'
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'val.txt')

In [ ]:
tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [ ]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings, CharacterEmbeddings

embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings : StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [ ]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)

In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('/content',
              learning_rate=0.5,
              mini_batch_size=32,
              max_epochs=150,
              )

2020-09-11 09:30:11,352 ----------------------------------------------------------------------------------------------------
2020-09-11 09:30:11,354 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=24, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-09-11 09:30:11,357 ----------------------------------------------------------------------------------------------------
2020-09-11 09:30:11,359 Corpus: "Corpus: 526 train + 36 dev + 31 test sentences"
2020-09-11 09:30:11,360 ----------------------------------------------------------------------------------------------------
2020-09-11 09:30:11,364 Parameters:
2020-09-11 09:30:11,365  - learning_rat

{'dev_loss_history': [5.717854976654053,
  7.014471054077148,
  6.267121315002441,
  4.367828369140625,
  7.306588172912598,
  3.1234371662139893,
  2.352734327316284,
  2.3950960636138916,
  2.601285934448242,
  2.621568202972412,
  2.6496293544769287,
  2.456434726715088,
  2.8066673278808594,
  2.356717109680176,
  2.2419240474700928,
  2.1843576431274414,
  2.249250888824463,
  2.1045773029327393,
  1.987985372543335,
  2.02628755569458,
  1.9274215698242188,
  1.9331698417663574,
  1.9181852340698242,
  1.9145869016647339,
  1.8926992416381836,
  1.829909324645996,
  1.877849817276001,
  1.8591759204864502,
  1.8752442598342896,
  1.8820831775665283,
  1.87209951877594,
  1.8655517101287842,
  1.860945701599121,
  1.8520863056182861,
  1.8610153198242188,
  1.8612797260284424,
  1.862546443939209,
  1.869424819946289,
  1.8706583976745605,
  1.8684940338134766,
  1.8697052001953125,
  1.8711341619491577,
  1.871195912361145,
  1.8707306385040283,
  1.8710083961486816,
  1.87022566

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
model = SequenceTagger.load('/content/final-model.pt')

text = '<redacted>'
sentence = Sentence(text)
model.predict(sentence)
out = {}
for entity in sentence.get_spans('ner'):
  out[entity.tag] = entity.text

2020-09-11 09:33:19,511 loading file /content/final-model.pt


In [ ]:
tags = ['EntityType','Recipient','Address',
        'GivenName','MiddleName','SurName','Household','Corporation',
        'StreetAddress','City','State','Zipcode','Country', 'Confidence']

PAD_TAIL = '<redacted>'

def household_patch(row):
  if row.EntityType=='Household':
    text = row.Household.split('&')[0] if '&' in row.Household else ' '.join(row.Household.split()[:2])
    text = parseit(text+PAD_TAIL)
    row['GivenName'] = text.loc[0,'GivenName']
    row['SurName'] = text.loc[0,'SurName']
    row['MiddleName'] = text.loc[0,'MiddleName']
  return row

def add_on(xx):
  # Address is the combination of street, city, state, zip and country
  xx['Address'] = xx['StreetAddress'] +' '+ xx['City'] +' '+ xx['State'] +' '+ xx['Zipcode'] +' '+ xx['Country']
  # default is person, if household column is not empty, then household, same for corporation
  xx['EntityType'] = 'Person'
  xx.loc[xx.Household!='','EntityType'] = 'Household'
  xx.loc[xx.Corporation!='','EntityType'] = 'Corporation'
  # default is full name of person, if entity is corporation ,then corporation name, same for household
  xx['Recipient'] = xx['SurName'] +' '+ xx['GivenName'] +' '+ xx['MiddleName']
  xx['Recipient'] = xx.apply(lambda row: row.Corporation if row.EntityType=='Corporation' else row.Recipient, axis=1)
  xx['Recipient'] = xx.apply(lambda row: row.Household if row.EntityType=='Household' else row.Recipient, axis=1)
  # adding household to name field patch
  xx = xx.apply(household_patch, axis=1)
  # converting to dictionary format
  xx = xx.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='any')
  xx = xx.T.to_dict()[0]
  # return the processed data
  return xx

def parseit(text):
  text = str(text).upper()
  text = re.sub('"','',text)
  text = re.sub(',','',text)
  text = ' '.join(text.split())
  output = {}
  sentence = Sentence(text)
  model.predict(sentence)
  for entity in sentence.get_spans('ner'):
    output[entity.tag] = entity.text
  df = pd.DataFrame(output, index=[0])
  dfx = pd.DataFrame(columns=tags)
  dfx = dfx.append(df).fillna('')
  dfx['Confidence'] = ','.join([f'[{x.tag} {str(round(x.score,2))}]'for x in sentence.get_spans('ner')])
  return dfx

def func(text):
  X = parseit(text)
  X = add_on(X)
  X = str(X)
  return X

In [ ]:
func('<redacted>')

In [ ]:
!pip install -q gradio
import gradio as gr
inputs = gr.inputs.Textbox(lines=3, label='Input')
outputs = gr.outputs.Textbox(label='Output')

In [ ]:
gr.Interface(fn=func, inputs=inputs, outputs=outputs).launch()

## API

In [ ]:
!pip install -q probablepeople
!pip install -q usaddress

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import  probablepeople as pp
import usaddress as ua

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()
%reload_ext autoreload
%autoreload 2
%reload_ext google.colab.data_table
%config InlineBackend.figure_format = 'retina'

plt.style.use('fivethirtyeight')
plt.style.use('seaborn-notebook')

In [ ]:
df = pd.read_pickle(os.path.join(path,'sample.p'))
data = df.oor.tolist()
data[:10]

In [ ]:
def patch_country(text):
  combos = [' US ',' USA ',' U.S. ',' U.S.A. ']
  cflagz = ''
  otext = text
  text = text[-10:]
  text = ' ' + text + ' '
  for i in combos:
    if i in text:
      cflagz = i
  text = otext[:-10] + re.sub(cflagz, '', text)
  cflagz = cflagz.strip()
  return text, cflagz

def patch_household(xz):
  xx = pd.DataFrame(pp.tag(xz)[0], index=[0])
  xx['Type'] = 'Person'
  if 'And' in xx.columns.tolist():
    xx = pd.DataFrame({'Household':xz}, index=[0])
    xx['Type'] = 'Household'
  return xx

tags = pp.LABELS
tags.extend(ua.LABELS)
tags = list(set(tags))
additionals = ['CountryName', 'Household', 'Type', 
               'NameConfidence', 'AddrConfidence', 'Text']
tags.extend(additionals)

dfx = pd.DataFrame(columns=tags)
dfx = dfx.loc[:,~dfx.columns.duplicated()]

In [ ]:
def process_text(text):
  text = re.sub(',','',text)
  text = ' '.join(text.split())
  text = re.sub('UNITED STATES OF AMERICA','US',text)
  text = re.sub('TEXAS','TX',text)
  return text

In [ ]:
def temp(text):
  text = process_text(text)
  text, cflag = patch_country(text)
  # address parsing
  try:
    df1 = pd.DataFrame(ua.tag(text)[0], index=[0])
    df1['AddrConfidence'] = ua.tag(text)[1]
  except:
    df1 = pd.DataFrame(ua.parse(text)).groupby(1).agg({0: lambda x: ' '.join(x)}).T
    df1['AddrConfidence'] = 'Error'
  # address to name linking
  try:
    xz = df1.Recipient.values[0]
  except:
    xz = text
  # name parsing
  try:
    df2 = pd.DataFrame(pp.tag(xz)[0], index=[0])
    df2['NameConfidence'] = pp.tag(xz)[1]
  except:
    df2 = pd.DataFrame(pp.parse(xz)).groupby(1).agg({0: lambda x: ' '.join(x)}).T
    df2['NameConfidence'] = 'Error'
  df2['Type'] = 'Person'
  # household name patch
  if 'And' in df2.columns.tolist():
    df2 = pd.DataFrame({'Household':xz}, index=[0])
    df2['Type'] = 'Household'
  # concatenation
  df = pd.concat([df1,df2], axis=1)
  df['Text'] = text  
  df['CountryName'] = cflag if cflag!='' else np.NaN
  dfx = pd.DataFrame(columns=tags)
  dfx = dfx.loc[:,~dfx.columns.duplicated()]
  df = dfx.append(df)
  df = df.fillna('')
  df.loc[df['CorporationName']!='', 'Type'] = 'Corporation'
  if (df['MiddleInitial'][0]=='') & (df['MiddleName'][0]!=''):
    gname, mname, sname = df.MiddleName, df.Surname, df.GivenName
    df['MiddleName'], df['Surname'], dfx['GivenName'] = mname[0], sname[0], gname[0]
  # elif (df['MiddleInitial'][0]=='') & (df['MiddleName'][0]==''):
  #   gname, sname = df.Surname, df.GivenName
  #   df['Surname'], df['GivenName'] = sname[0], gname[0]
  return df

In [ ]:
tagmap = pd.read_excel(os.path.join(path,'tag_map.xlsx')).reset_index()
tagmap.columns = ['_'.join(col.split()) for col in tagmap.columns]
tagmap
tagmap = pd.melt(tagmap, id_vars=['index'], value_vars=['FIRSTNAME','MIDDLENAME','LASTNAME',
                                                        'OTHERNAME','HOUSEHOLD','CORPORATION',
                                                        'STREET_ADDRESS', 'CITY', 'STATE', 'PINCODE', 
                                                        'COUNTRY', 'META', 'OTHER']).dropna()[['variable','value']].set_index('value').to_dict()['variable']

In [ ]:
def custom_patch(text):
  text = re.sub(' DALLA S ', ' DALLAS ', text)
  text = re.sub(' PRAIRI E ', ' PRAIRIE ', text)
  text = re.sub(' EL PAS O ', ' EL PASO ', text)
  text = re.sub(' SAN ANT ONIO ', ' SAN ANTONIO ', text)
  text = re.sub(' THE WOODL ANDS ', ' THE WOODLANDS ', text)
  text = re.sub(' FORT WORT H ', ' FORT WORTH ', text)
  text = re.sub(' ARLINGTO N ', ' ARLINGTON ', text)
  text = re.sub(' SOCORR O ', ' SOCORRO ', text)
  text = re.sub(' HURS T ', ' HURST ', text)
  text = re.sub(' HOLLYWOO D ', ' HOLLYWOOD ', text)
  return text

In [ ]:
def oor_api(text):
  texto = text
  xx = temp(text)
  xx.columns = xx.columns.to_series().map(tagmap)
  xx = xx.drop('META', axis=1)
  xx = xx.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='any')
  try:
    xx['PINCODE'] = xx['PINCODE'].apply(lambda x: re.sub(' ','',x))
  except:
    pass
  xx = xx.T
  xx = xx.reset_index()
  xx.index = xx.index.map(str)
  xx = ' '+ xx + ' '
  xx[0] = xx[0].apply(custom_patch)
  xx = xx.to_dict()
  label2tag = {str(v): ' '+str(k)+' ' for k, v in xx[0].items()}
  text = process_text(text)
  text = ' ' + text + ' '
  for key, value in label2tag.items():
    text = text.replace(key, value)
  x = pd.DataFrame({'entity':text.split(), 'tag':text.split()})
  x = x.replace({'entity': xx[0], 'tag': xx['index']})
  x['entity'] = x.groupby(['tag'])['entity'].transform(lambda x: ' '.join(x))
  x = x.drop_duplicates()
  x['tag'] = x['tag'].apply(process_text)
  x['entity'] = x['entity'].apply(process_text)
  x = x.append(pd.Series({'tag':'TEXT','entity':texto}), ignore_index=True)
  x = x.set_index('tag')
  x = x.to_dict()['entity']
  return x

In [ ]:
# zz = data[5]
# oor_api(zz)

In [ ]:
oor_cols = ['TEXT','FIRSTNAME','MIDDLENAME','LASTNAME', 'OTHERNAME', 'HOUSEHOLD', 
            'CORPORATION', 'STREET_ADDRESS', 'CITY', 'STATE', 'PINCODE', 'COUNTRY']
dfx = pd.DataFrame(columns=oor_cols)
for idx, text in tqdm(enumerate(data)):
  dfx = dfx.append(pd.DataFrame(oor_api(text), index=[idx]))
dfx = dfx.fillna('')
dfx = dfx[oor_cols]

1000it [01:03, 15.75it/s]


In [ ]:
dfx = dfx.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TEXT            997 non-null    object
 1   FIRSTNAME       403 non-null    object
 2   MIDDLENAME      155 non-null    object
 3   LASTNAME        324 non-null    object
 4   OTHERNAME       9 non-null      object
 5   HOUSEHOLD       199 non-null    object
 6   CORPORATION     364 non-null    object
 7   STREET_ADDRESS  976 non-null    object
 8   CITY            870 non-null    object
 9   STATE           954 non-null    object
 10  PINCODE         838 non-null    object
 11  COUNTRY         288 non-null    object
dtypes: object(12)
memory usage: 101.6+ KB


In [ ]:
dfx.to_csv('x.csv')

In [ ]:
x = '<redacted>'
temp(x).T.replace('',np.nan).dropna()

In [ ]:
oor_api(x)

## Deliverables

In [ ]:
# !pip install cookiecutter
# !cookiecutter https://github.com/drivendata/cookiecutter-data-science

In [ ]:
imports _requirements.txt_
!pip install flair

In [ ]:
# utils

import pandas as pd

tags = ['EntityType','Recipient','Address',
        'GivenName','MiddleName','SurName','Household','Corporation',
        'StreetAddress','City','State','Zipcode','Country', 'Confidence']
name_tags = ['EntityType','Recipient', 'GivenName','MiddleName','SurName','Household','Corporation']
addr_tags = ['Address','StreetAddress','City','State','Zipcode','Country']

PAD_TAIL = '<redacted>'
PAD_HEAD = '<redacted>'

def household_patch(row):
  if row.EntityType=='Household':
    text = row.Household.split('&')[0] if '&' in row.Household else ' '.join(row.Household.split()[:2])
    text = parseit(text+PAD_TAIL)
    row['GivenName'] = text.loc[0,'GivenName']
    row['SurName'] = text.loc[0,'SurName']
    row['MiddleName'] = text.loc[0,'MiddleName']
  return row

def add_on(xx):
  # Address is the combination of street, city, state, zip and country
  xx['Address'] = xx['StreetAddress'] +' '+ xx['City'] +' '+ xx['State'] +' '+ xx['Zipcode'] +' '+ xx['Country']
  # default is person, if household column is not empty, then household, same for corporation
  xx['EntityType'] = 'Person'
  xx.loc[xx.Household!='','EntityType'] = 'Household'
  xx.loc[xx.Corporation!='','EntityType'] = 'Corporation'
  # default is full name of person, if entity is corporation ,then corporation name, same for household
  xx['Recipient'] = xx['SurName'] +' '+ xx['GivenName'] +' '+ xx['MiddleName']
  xx['Recipient'] = xx.apply(lambda row: row.Corporation if row.EntityType=='Corporation' else row.Recipient, axis=1)
  xx['Recipient'] = xx.apply(lambda row: row.Household if row.EntityType=='Household' else row.Recipient, axis=1)
  # adding household to name field patch
  xx = xx.apply(household_patch, axis=1)
  # converting to dictionary format
  xx = xx.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='any')
  # return the processed data
  return xx

In [ ]:
# pred

import re
import numpy as np
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger

model = SequenceTagger.load('models/final-model.pt')

def _parse(text):
  text = str(text).upper()
  text = re.sub('"','',text)
  text = re.sub(',','',text)
  text = ' '.join(text.split())
  output = {}
  sentence = Sentence(text)
  model.predict(sentence)
  for entity in sentence.get_spans('ner'):
    output[entity.tag] = entity.text
  df = pd.DataFrame(output, index=[0])
  dfx = pd.DataFrame(columns=tags)
  dfx = dfx.append(df).fillna('')
  dfx['Confidence'] = ','.join([f'[{x.tag} {str(round(x.score,2))}]'for x in sentence.get_spans('ner')])
  return dfx

2020-09-11 13:02:40,896 loading file models/final-model.pt


In [ ]:
def _predict(text):
  X = _parse(text)
  X = add_on(X)
  X = X.T.to_dict()[0]
  return X

def _predict_name(text):
  text = text + PAD_TAIL
  X = _parse(text)
  X = add_on(X)
  rqd_cols = list(set(X.columns) & set(name_tags))
  X = X[rqd_cols]
  X = X.T.to_dict()[0]
  return X

def _predict_address(text):
  text = PAD_HEAD + text
  X = _parse(text)
  X = add_on(X)
  rqd_cols = list(set(X.columns) & set(addr_tags))
  X = X[rqd_cols]
  X = X.T.to_dict()[0]
  return X

In [ ]:
# app
_predict('<redacted>')

---

In [ ]:
%%writefile src/app.py

import re
import time
import numpy as np
import pandas as pd
from pathlib import Path
from flair.data import Sentence
from flair.models import SequenceTagger

from utils import *
from model import loadmodel

path = Path(__file__)
_PPATH = str(path.parents[1])+'/'

start_time = time.time()
model = loadmodel.finalmodel
print("---Encoder--- %s seconds ---" % (time.time() - start_time))

def _parse(text):
  text = str(text).upper()
  text = re.sub('"','',text)
  text = re.sub(',','',text)
  text = ' '.join(text.split())
  output = {}
  sentence = Sentence(text)
  model.predict(sentence)
  for entity in sentence.get_spans('ner'):
    output[entity.tag] = entity.text
  df = pd.DataFrame(output, index=[0])
  dfx = pd.DataFrame(columns=tags)
  dfx = dfx.append(df).fillna('')
  dfx['Confidence'] = ','.join([f'[{x.tag} {str(round(x.score,2))}]'for x in sentence.get_spans('ner')])
  return dfx

def _predict(text):
  X = _parse(text)
  X = add_on(X)
  X = X.T.to_dict()[0]
  return X

def _predict_name(text):
  text = text + PAD_TAIL
  X = _parse(text)
  X = add_on(X)
  rqd_cols = list(set(X.columns) & set(name_tags))
  X = X[rqd_cols]
  X = X.T.to_dict()[0]
  return X

def _predict_address(text):
  text = PAD_HEAD + text
  X = _parse(text)
  X = add_on(X)
  rqd_cols = list(set(X.columns) & set(addr_tags))
  X = X[rqd_cols]
  X = X.T.to_dict()[0]
  return X

########## FLASK API ##########

from flask import Flask, request, jsonify, send_file
import json

app = Flask(__name__)

@app.route("/oor", methods=["POST"])
def parse_oor():
  req_data = request.get_json()
  text = req_data['query']
  preds = _predict(text)
  return jsonify(preds)

@app.route("/name", methods=["POST"])
def parse_name():
  req_data = request.get_json()
  text = req_data['query']
  preds = _predict_name(text)
  return jsonify(preds)

@app.route("/address", methods=["POST"])
def parse_address():
  req_data = request.get_json()
  text = req_data['query']
  preds = _predict_address(text)
  return jsonify(preds)

if __name__ == "__main__":
    app.run(port=5000, debug=True)

Overwriting src/app.py


In [ ]:
!pip install -q pyngrok
from pyngrok import ngrok
!ngrok authtoken <redacted>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
ngrok.kill()
public_url = ngrok.connect(port='5000'); public_url

'http://c1b4cabd8e5b.ngrok.io'

In [ ]:
!python src/app.py

2020-09-16 16:05:27.961051: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 16:05:30,365 loading file /content/p28118/models/final-model.pt
---Encoder--- 1.430511474609375e-06 seconds ---
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
2020-09-16 16:05:35.186608: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 16:05:37,169 loading file /content/p28118/models/final-model.pt
---Encoder--- 1.1920928955078125e-06 seconds ---
 * Debugger is active!
 * Debugger PIN: 210-465-697


## Pipeline

In [ ]:
!pip install -q probablepeople
!pip install -q usaddress

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import  probablepeople as pp
import usaddress as ua

import shutil
# shutil.make_archive(os.path.join(path,'customNER'),'zip','/content/customNER')
shutil.unpack_archive(os.path.join(path,'customNER.zip'), '/content/customNER', 'zip')

In [ ]:
df = pd.read_pickle(os.path.join(path,'sample.p'))
data = df.oor.tolist()
data[100:110]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 3605089 to 2516200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     1000 non-null   object
 1   oor     1000 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


In [ ]:
import spacy
model = spacy.load('/content/customNER')

In [ ]:
text1 = '<redacted>'

def name_parsing(text):
  try:
    text = pp.tag(text)
    textdf = pd.DataFrame(text[0], index=[0])
    textdf['NameType'] = text[1]
    textdf['NameConfidence'] = 'Tag'
    textdf = textdf.T.to_dict()[0]
  except:
    textdf = pp.parse(text)
    textdf = pd.DataFrame(textdf).groupby(1).agg({0: lambda x: ' '.join(x)}).T
    textdf['NameType'] = 'Ambiguous'
    textdf['NameConfidence'] = 'Parse'
    textdf = textdf.T.to_dict()[0]
  return textdf

name_parsing(text3)

In [ ]:
def preprocess(text):
  text = re.sub(',','',text)
  text = ' '.join(text.split())
  text = re.sub('UNITED STATES OF AMERICA','US',text)
  return text

In [ ]:
text1 = '<redacted>'

def address_parsing(text):
  try:
    text = preprocess(text)
    text = ua.tag(text)
    textdf = pd.DataFrame(text[0], index=[0])
    textdf['AddressType'] = text[1]
    textdf['AddrConfidence'] = 'Tag'
    textdf = textdf.T.to_dict()[0]
  except:
    textdf = ua.parse(text)
    textdf = pd.DataFrame(textdf).groupby(1).agg({0: lambda x: ' '.join(x)}).T
    textdf['AddressType'] = 'Ambiguous'
    textdf['AddrConfidence'] = 'Parse'
    textdf = textdf.T.to_dict()[0]
  return textdf

xx = address_parsing(text2); xx

In [ ]:
list(xx.keys())

['USPSBoxType',
 'USPSBoxID',
 'PlaceName',
 'StateName',
 'ZipCode',
 'ZipPlus4',
 'CountryName',
 'AddressType',
 'AddrConfidence']

In [ ]:
text1 = '<redacted>'

def full_parsing(text):
  doc = model(text)
  ners = {ent.label_:ent.text for ent in doc.ents}
  name = address = {}
  try:
    address = address_parsing(ners['Address'])
  except:
    pass
  try:
    name = name_parsing(ners['Name'])
  except:
    pass
  address.update(name)
  try:
    del address['Recipient']
  except:
    pass
  if 'And' in list(address.keys()):
    xx = pd.DataFrame({'Household':xz}, index=[0])
    xx['Type'] = 'Household'
  return address

xx = full_parsing(text2); xx

In [ ]:
tags = pp.LABELS
tags.extend(ua.LABELS)
tags = list(set(tags))
additionals = ['Text', 'AddressType','AddrConfidence','NameType','NameConfidence']
tags.extend(additionals)
dfx = pd.DataFrame(columns=tags)
dfx = dfx.loc[:,~dfx.columns.duplicated()]
for idx, text in tqdm(enumerate(data)):
  dfx = dfx.append(full_parsing(text), ignore_index=True)
  dfx.loc[idx, 'Text'] = text

In [ ]:
sorted_collist = dfx.isna().sum().sort_values().index
dfx = dfx[sorted_collist]

In [ ]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Text                               1000 non-null   object
 1   NameConfidence                     994 non-null    object
 2   NameType                           994 non-null    object
 3   AddrConfidence                     989 non-null    object
 4   AddressType                        989 non-null    object
 5   PlaceName                          967 non-null    object
 6   StateName                          967 non-null    object
 7   AddressNumber                      933 non-null    object
 8   StreetName                         926 non-null    object
 9   StreetNamePostType                 864 non-null    object
 10  ZipCode                            857 non-null    object
 11  GivenName                          627 non-null    object
 12  Surname

In [ ]:
dfx0 = dfx.dropna(how='all').reset_index(drop=True); print(dfx0.info())
dfx1 = dfx.dropna(how='all', axis=1).dropna(how='all').reset_index(drop=True); dfx1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Text                               1000 non-null   object
 1   NameConfidence                     994 non-null    object
 2   NameType                           994 non-null    object
 3   AddrConfidence                     989 non-null    object
 4   AddressType                        989 non-null    object
 5   PlaceName                          967 non-null    object
 6   StateName                          967 non-null    object
 7   AddressNumber                      933 non-null    object
 8   StreetName                         926 non-null    object
 9   StreetNamePostType                 864 non-null    object
 10  ZipCode                            857 non-null    object
 11  GivenName                          627 non-null    object
 12  Surname

In [ ]:
# dfx0.to_csv('dfx0.csv')
dfx1.to_csv('dfx1.csv')

In [ ]:
def func(text):
  text = str(text)
  output = full_parsing(text)
  return str(output)

txt = '<redacted>'
func(txt)

In [ ]:
!pip install -q gradio
import gradio as gr

In [ ]:
gr.Interface(fn=func, inputs="text", outputs="text").launch()